In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from tonic.models.vic import (netcdf2vic, compare_soil_params, grid_params, 
                              ncparam2ascii, vic2netcdf)
import numpy as np
import argparse
import subprocess
import os
from tonic import version
from scipy.spatial import cKDTree
from tonic.io import read_netcdf
from netCDF4 import Dataset, default_fillvals


In [4]:
# import NetCDF params file
direc = '/Users/diana/Dropbox/UW/Research/rasm/25_km'
ncparams_filename = 'vic_params_wr50a_vic5.0.dev_20160328.nc'
ncparams_file_name_with_snowbands = 'vic_params_wr50a_vic5.0.dev_joe_8312017.nc'
nc_params_25km = "params_25km_full.nc"
ncparams = os.path.join(direc, ncparams_filename)
ncparams_with_snowbands = os.path.join(direc, ncparams_file_name_with_snowbands)
ncparams_25km = os.path.join(direc, nc_params_25km)
soil_file = os.path.join(direc, 'soil_params.txt')
veg_file = os.path.join(direc, 'veg_params.txt')
snow_file = os.path.join(direc, 'snow_params.txt')

In [5]:
def rasm_soil(data, soil_file):
    """Write VIC formatted soil parameter file"""

    message = """
*** ---------------------------------------------------------------------- ***
Notes about RASM soil parameter file generations:
 - To fill in missing grid cells 'mask' variable must be the same as the
   domain file mask.
 - Inactive grid cells will have a dummy line printed for all variables except
   the lons/lats.
 - Any grid cells with nans will be copied from the previous line without nans
   or FILL_VALUEs.
*** --------------------------------------------------------------------- ***\n
    """

    print(message)
    
    FILL_VALUE = -9999

    # ---------------------------------------------------------------- #
    c = grid_params.Cols(nlayers=3)
    f = grid_params.Format(nlayers=3)

    c.soil_param['Nveg'] = np.array([0])
    f.soil_param['Nveg'] = '%1i'

    if 'mask' in data:
        mask = 'mask'
    elif 'run_cell' in data:
        mask = 'run_cell'
    else: 
        raise KeyError("Land mask must exist in NetCDF file.")
    
    numcells = data[mask].size

    arrayshape = (numcells, 1 + np.max([np.max(cols) for v, cols in
                                        c.soil_param.items()]))
    soil_params = np.empty(arrayshape)
    dtypes = ['%1i'] * arrayshape[1]
    # ---------------------------------------------------------------- #

    # ---------------------------------------------------------------- #
    # find nearest real grid cell for all grid cells where frozen soil mask is
    # active
    # This is needed because the RASM mask is often different than existing
    # datasets
    print('Finding/filling nearest neighbors for all variables based on '
          'fs_active mask')

    # real mask (from input dataset)
    ry, rx = np.nonzero(data['fs_active'])

        # fill mask (we will fill all of these grid cells with their nearest real
    # value)
    my_mask = np.zeros(data['fs_active'].shape, dtype=int)
    my_mask[ry, rx] = 1

    fy, fx = np.nonzero(my_mask == 0)

    # Find nearest real grid cell
    combined = np.dstack(([data['yc'][ry, rx], data['xc'][ry, rx]]))[0]
    points = list(np.vstack((data['yc'][fy, fx],
                             data['xc'][fy, fx])).transpose())
    mytree = cKDTree(combined)
    dist, inds = mytree.query(points, k=1)

    # loop over all variables and fill in values
    for var in c.soil_param:
        if var not in ['run_cell', 'grid_cell', 'lats', 'lons', 'fs_active',
                       'Nveg']:
            # unmask the variable
            data[var] = np.array(data[var])
            # fill with nearest data
            if data[var].ndim == 2:
                data[var][fy, fx] = data[var][ry[inds], rx[inds]]
            elif data[var].ndim == 3:
                data[var][:, fy, fx] = data[var][:, ry[inds], rx[inds]]
    # ---------------------------------------------------------------- #

    # ---------------------------------------------------------------- #
    # This is the domain mask
    yinds, xinds = np.nonzero(data[mask] == 1)
    ymask, xmask = np.nonzero(data[mask] != 1)
    # ---------------------------------------------------------------- #

    # ---------------------------------------------------------------- #
    # Fix problematic avg_T values
    print('Finding/filling nearest neighbors for avg_T\n')

    # real mask (from input dataset)
    ry, rx = np.nonzero((data['avg_T'] > -50) & (data['avg_T'] < 99))

        # fill mask (we will fill all of these grid cells with their nearest
    # real value)
    my_mask = np.zeros(data['avg_T'].shape)
    my_mask[ry, rx] = 1

    fy, fx = np.nonzero(my_mask == 0)

    # Find nearest real grid cell
    if len(fy) > 0:
        combined = np.dstack(([data['yc'][ry, rx], data['xc'][ry, rx]]))[0]
        points = list(np.vstack((data['yc'][fy, fx],
                                 data['xc'][fy, fx])).transpose())
        mytree = cKDTree(combined)
        dist, inds = mytree.query(points, k=1)

        data['avg_T'] = np.array(data['avg_T'])
        data['avg_T'][fy, fx] = data['avg_T'][ry[inds], rx[inds]]
    # ---------------------------------------------------------------- #

    # ---------------------------------------------------------------- #
    # For rasm, all cols are shifted one to right to make room for nveg in
    # col 0
    i = -1
    for var, cols in c.soil_param.items():
        for col in cols:
            dtypes[col] = f.soil_param[var]

    for (y, x), maskval in np.ndenumerate(data[mask]):
        # advance the row count
        i += 1

        # put real data
        for var in c.soil_param:
            cols = c.soil_param[var]
            if data[var].ndim == 2:
                soil_params[i, cols] = data[var][y, x]
            elif data[var].ndim == 3:
                for j, col in enumerate(cols):
                    soil_params[i, col] = data[var][j, y, x]

        # write the grid cell number
        soil_params[i, 2] = i + 1
    # ---------------------------------------------------------------- #
    # ---------------------------------------------------------------- #
    # Write phi_s
    soil_params[:, c.soil_param['phi_s']] = -999
    # ---------------------------------------------------------------- #

    # ---------------------------------------------------------------- #
    # Write nveg
    soil_params[:, 0] = data['Nveg'][:, :].flatten()
    # ---------------------------------------------------------------- #

    # ---------------------------------------------------------------- #
    # Set all grid cells to run
    soil_params[:, 1] = 1  # run
    soil_params[:, -1] = 1  # run with frozen soils on
    # ---------------------------------------------------------------- #

    # ---------------------------------------------------------------- #
    # rewrite the lons/lats columns
    soil_params[:, 3] = data['lats'].flatten()
    soil_params[:, 4] = data['lons'].flatten()

    assert soil_params[-1, 3] == data['lats'][y, x]
    assert soil_params[-1, 4] == data['lons'][y, x]
    # ---------------------------------------------------------------- #

    # ---------------------------------------------------------------- #
    # check for nans
    # sometimes RASM has land grid cells that the input file does not.
    # In this case, we will use the previous lnd grid cell
    if np.isnan(soil_params.sum()):
        bad_cells = []
        replacements = []
        for i, cell in enumerate(soil_params):
            if np.isnan(np.sum(cell)):
                bad_cells.append(i)
                j = i
                while True:
                    j -= 1
                    if (FILL_VALUE not in soil_params[j, :]) and \
                            not np.isnan(np.sum(soil_params[j, :])):
                        soil_params[i, 5:] = soil_params[j, 5:]
                        replacements.append(j)
                        break
        print('Fixed {0} bad cells'.format(len(bad_cells)))
        print('Example: {0}-->{1}'.format(bad_cells[0], replacements[0]))
    # ---------------------------------------------------------------- #
    # ---------------------------------------------------------------- #
    # Print summary of variables
    for var, cols in c.soil_param.items():
        print('{0: <12}--> min: {1:<09.3f}, max: {2:<09.3f}, mean:'
              ' {3:<09.3f}'.format(var,
                                   soil_params[:, cols].min(),
                                   soil_params[:, cols].max(),
                                   soil_params[:, cols].mean()))
    # ---------------------------------------------------------------- #

    # ---------------------------------------------------------------- #
    # Finish up
    assert soil_params[-1, 3] == data['lats'][y, x]
    assert soil_params[-1, 4] == data['lons'][y, x]
    # ---------------------------------------------------------------- #

    # ---------------------------------------------------------------- #
    # Write the file
    print('writing soil parameter file: {0}'.format(soil_file))

    np.savetxt(soil_file, soil_params, fmt=dtypes)

    print('done RASM with soil')
    # ---------------------------------------------------------------- #

    return

In [6]:
def soil(data, xinds, yinds, soil_file):
    """Write VIC formatted soil parameter file"""
    c = grid_params.cols(nlayers=3)
    f = grid_params.format(nlayers=3)

    arrayshape = (len(xinds), 1 + np.max([np.max(c.soil_param[var])
                                          for var in c.soil_param]))
    soil_params = np.zeros(arrayshape)
    dtypes = [0] * arrayshape[1]

    for var in c.soil_param:
        if data[var].ndim == 2:
            soil_params[:, c.soil_param[var]] = np.atleast_2d(
                data[var][yinds, xinds]).transpose()
        elif data[var].ndim == 3:
            soil_params[:, c.soil_param[var]] = np.atleast_2d(
                data[var][:, yinds, xinds]).transpose()
        for col in c.soil_param[var]:
            dtypes[col] = f.soil_param[var]

    np.savetxt(soil_file, soil_params, fmt=dtypes)

    print('finished writing soil parameter file: {0}'.format(soil_file))

    return

In [7]:
def snow(data, xinds, yinds, snow_file):
    """Write VIC formatted snowband parameter file"""
    try:
        snow_bands = data['AreaFract'].shape[0]
    except:
        snow_bands = 5

    c = grid_params.Cols(snow_bands=snow_bands)
    f = grid_params.Format(snow_bands=snow_bands)

    arrayshape = (len(xinds), 1 + np.max([np.max(c.snow_param[var])
                                          for var in c.snow_param]))
    snow_params = np.zeros(arrayshape)
    dtypes = [0] * arrayshape[1]

    for var in c.snow_param:
        print(var)
        if data[var].ndim == 2:
            snow_params[:, c.snow_param[var]] = np.atleast_2d(
                data[var][yinds, xinds]).transpose()
        elif data[var].ndim == 3:
            snow_params[:, c.snow_param[var]] = np.atleast_2d(
                data[var][:, yinds, xinds]).transpose()
        for col in c.snow_param[var]:
            dtypes[col] = f.snow_param[var]

    np.savetxt(snow_file, snow_params, fmt=dtypes)

    print('finished writing snow parameter file: {0}'.format(snow_file))

    return

In [15]:
def veg(data, xinds, yinds, veg_file, rootzones=3, global_lai=True):
    """Write VIC formatted veg parameter file"""

    print('writing veg parameter file: {0}'.format(veg_file))

    # counter for bad grid cells
    count = 0

    f = open(veg_file, 'w')

    for y, x in zip(yinds, xinds):
        gridcell = int(data['gridcell'][y, x])
        n_veg = int(data['Nveg'][y, x])
        cv = data['Cv'][:, y, x]
        veg_class = np.nonzero(cv)[0]

        if not len(veg_class) == n_veg:
            count += 1

        line1 = str(gridcell) + ' ' + str(n_veg) + '\n'
        f.write(line1)
        if n_veg > 0:
            for j in veg_class:
                line2 = [str(j + 1)]
                line2.append(str(cv[j]))
                for k in range(rootzones):
                    line2.append(str(data['root_depth'][j, k, y, x]))
                    line2.append(str(data['root_fract'][j, k, y, x]))
                line2.append('\n')
                f.write(' '.join(line2))
                if global_lai:
                    line3 = []
                    for m in range(12):
                        line3.append(str(data['LAI'][j, m, y, x]))
                    line3.append('\n')
                    f.write(' '.join(line3))
    f.close()

    print('{0} grid cells have unequal veg_classes'.format(count))
    print('finished writing veg parameter file: {0}'.format(veg_file))

    return

In [9]:
def subset(param_file, upleft=False, lowright=False, outfiles=1,
           soil_file=False, snow_file=False,
           veg_file=False, project=None,
           nijssen2arno=False):
    print("in subset function")
    data, attributes = read_netcdf(param_file)

    if nijssen2arno:
        import NIJSSEN2001_to_ARNO
        data = NIJSSEN2001_to_ARNO.convert(data)
        
    if 'mask' in data:
        mask = 'mask'
    elif 'run_cell' in data:
        mask = 'run_cell'
    else: 
        raise KeyError("Land mask must exist in NetCDF file.")

    if project:
        print('Project Configuration {0}'.format(project))
        if project == 'RASM':
            outfiles = 1 
            cells, yinds, xinds = find_gridcells(data[mask])
            rasm_soil(data, soil_file)
        else:
            raise ValueError('Unknown project configuration')
        return

    else:
        cells, yinds, xinds = find_gridcells(data[mask])

    # write snow and veg files
    if veg_file:
        rootzones = data['root_depth'].shape[1]
        print("writing veg file")
        veg(data, xinds, yinds, veg_file, rootzones=rootzones, global_lai=True)
        print("wrote veg file")
    if snow_file:
        print("write snow file")
        snow(data, xinds, yinds, snow_file)
        print("wrote snow file")

    if (upleft and lowright):
        inds = ((yinds < upleft[0]) and
                (yinds > lowright[0]) and
                (xinds < lowright[1]) and
                (xinds > upleft[1]))
        yinds = yinds[inds]
        xinds = xinds[inds]

    filesize = np.ceil(cells / outfiles)
    for i in range(outfiles):
        start = i * filesize
        end = i * filesize + filesize
        if end > cells:
            end = cells
        if outfiles > 1:
            out_file = '{0}_{1}.txt'.format(soil_file,
                                            str(i).zfill(len(str(outfiles))))
        else:
            out_file = '{0}.txt'.format(soil_file)
        soil(data, xinds[start:end], yinds[start:end], out_file)

    return

In [10]:
def find_gridcells(mask):
    """ Find number of grid cells in mask"""
    cells = np.sum(mask)

    xinds, yinds = np.nonzero(mask > 0)

    return cells, xinds, yinds

make ASCII parameters at 50km resolution to be sure it works

In [10]:
# define fill values (for some reason doing this in grid_params.py isn't working)
NC_DOUBLE = 'f8'
FILLVALUE_F = default_fillvals[NC_DOUBLE]

subset(ncparams, 
       upleft=False, 
       lowright=False, 
       soil_file=soil_file,
       snow_file=snow_file,
       veg_file=veg_file,
       project='RASM',
       nijssen2arno=False)

in subset function
Reading input data variables:  odict_keys(['yc', 'xc', 'xv', 'yv', 'mask', 'layer', 'run_cell', 'gridcell', 'lats', 'lons', 'infilt', 'Ds', 'Dsmax', 'Ws', 'c', 'expt', 'Ksat', 'phi_s', 'init_moist', 'elev', 'depth', 'avg_T', 'dp', 'bubble', 'quartz', 'bulk_density', 'soil_density', 'off_gmt', 'Wcr_FRACT', 'Wpwp_FRACT', 'rough', 'snow_rough', 'annual_prec', 'resid_moist', 'fs_active', 'veg_class', 'veg_descr', 'month', 'Nveg', 'Cv', 'root_zone', 'root_depth', 'root_fract', 'LAI', 'overstory', 'rarc', 'rmin', 'wind_h', 'RGL', 'rad_atten', 'wind_atten', 'trunk_ratio', 'albedo', 'veg_rough', 'displacement']) from file: /Users/diana/Dropbox/UW/Research/rasm/25_km/vic_params_wr50a_vic5.0.dev_20160328.nc
Project Configuration RASM

*** ---------------------------------------------------------------------- ***
Notes about RASM soil parameter file generations:
 - To fill in missing grid cells 'mask' variable must be the same as the
   domain file mask.
 - Inactive grid cells 

have to make the veg and snow files separately because if `RASM == True` then those files aren't made 

In [11]:
data, attributes = read_netcdf(ncparams_with_snowbands)
cells, yinds, xinds = find_gridcells(data['mask'])
rootzones = data['root_depth'].shape[1]
# veg(data, xinds, yinds, veg_file, rootzones=rootzones, global_lai=True)
# snow(data, xinds, yinds, snow_file)

Reading input data variables:  odict_keys(['yc', 'xc', 'xv', 'yv', 'mask', 'run_cell', 'gridcell', 'lats', 'lons', 'infilt', 'Ds', 'Dsmax', 'Ws', 'c', 'expt', 'Ksat', 'phi_s', 'init_moist', 'elev', 'depth', 'avg_T', 'dp', 'bubble', 'quartz', 'bulk_density', 'soil_density', 'off_gmt', 'Wcr_FRACT', 'Wpwp_FRACT', 'rough', 'snow_rough', 'annual_prec', 'resid_moist', 'fs_active', 'cellnum', 'AreaFract', 'elevation', 'Pfactor', 'month', 'Nveg', 'Cv', 'root_depth', 'root_fract', 'LAI', 'overstory', 'rarc', 'rmin', 'wind_h', 'RGL', 'rad_atten', 'wind_atten', 'trunk_ratio', 'snow_albedo', 'albedo', 'veg_rough', 'displacement']) from file: /Users/diana/Dropbox/UW/Research/rasm/25_km/vic_params_wr50a_vic5.0.dev_joe_8312017.nc


now derive 25 km parameters from snowbands 25km NetCDF parameter file (otherwise can't produce snow bands file) 

(NOTE: this was remapped via NN-mapping from 50km to 25km, see `/Users/diana/Dropbox/UW/Research/rasm/25_km/regrid_params.sh` )

In [11]:
NC_DOUBLE = 'f8'
FILLVALUE_F = default_fillvals[NC_DOUBLE]

In [12]:
# 25 km parameters file names 
soil_file = os.path.join(direc, 'soil_params_25km.txt')
veg_file = os.path.join(direc, 'veg_params_25km.txt')
snow_file = os.path.join(direc, 'snow_params_25km.txt')

RASM soil file 

In [13]:
# ncparams_25km is the remapped 
subset(ncparams_25km, 
       upleft=False, 
       lowright=False, 
       soil_file=soil_file,
       snow_file=False,
       veg_file=False,
       project='RASM',
       nijssen2arno=False)

in subset function
Reading input data variables:  odict_keys(['veg_class', 'month', 'root_zone', 'snow_band', 'nlayer', 'yc', 'xc', 'run_cell', 'mask', 'xv', 'yv', 'gridcell', 'lats', 'lons', 'infilt', 'Ds', 'Dsmax', 'Ws', 'c', 'expt', 'Ksat', 'phi_s', 'init_moist', 'elev', 'depth', 'avg_T', 'dp', 'bubble', 'quartz', 'bulk_density', 'soil_density', 'off_gmt', 'Wcr_FRACT', 'Wpwp_FRACT', 'rough', 'snow_rough', 'annual_prec', 'resid_moist', 'fs_active', 'cellnum', 'AreaFract', 'elevation', 'Pfactor', 'Nveg', 'Cv', 'root_depth', 'root_fract', 'LAI', 'overstory', 'rarc', 'rmin', 'wind_h', 'RGL', 'rad_atten', 'wind_atten', 'trunk_ratio', 'snow_albedo', 'albedo', 'veg_rough', 'displacement']) from file: /Users/diana/Dropbox/UW/Research/rasm/25_km/params_25km_full.nc
Project Configuration RASM

*** ---------------------------------------------------------------------- ***
Notes about RASM soil parameter file generations:
 - To fill in missing grid cells 'mask' variable must be the same as the


## NOTE: even though `infilt`, `Nveg` and `run_cell` look weird in this chart, I just checked all of them in the NetCDF file and they're good 

veg and snow files

In [16]:
data, attributes = read_netcdf(ncparams_25km)
cells, yinds, xinds = find_gridcells(data['run_cell'])
rootzones = data['root_depth'].shape[1]
veg(data, xinds, yinds, veg_file, rootzones=3, global_lai=True)

Reading input data variables:  odict_keys(['veg_class', 'month', 'root_zone', 'snow_band', 'nlayer', 'yc', 'xc', 'run_cell', 'mask', 'xv', 'yv', 'gridcell', 'lats', 'lons', 'infilt', 'Ds', 'Dsmax', 'Ws', 'c', 'expt', 'Ksat', 'phi_s', 'init_moist', 'elev', 'depth', 'avg_T', 'dp', 'bubble', 'quartz', 'bulk_density', 'soil_density', 'off_gmt', 'Wcr_FRACT', 'Wpwp_FRACT', 'rough', 'snow_rough', 'annual_prec', 'resid_moist', 'fs_active', 'cellnum', 'AreaFract', 'elevation', 'Pfactor', 'Nveg', 'Cv', 'root_depth', 'root_fract', 'LAI', 'overstory', 'rarc', 'rmin', 'wind_h', 'RGL', 'rad_atten', 'wind_atten', 'trunk_ratio', 'snow_albedo', 'albedo', 'veg_rough', 'displacement']) from file: /Users/diana/Dropbox/UW/Research/rasm/25_km/params_25km_full.nc
writing veg parameter file: /Users/diana/Dropbox/UW/Research/rasm/25_km/veg_params_25km.txt
35646 grid cells have unequal veg_classes
finished writing veg parameter file: /Users/diana/Dropbox/UW/Research/rasm/25_km/veg_params_25km.txt


In [17]:
snow(data, xinds, yinds, snow_file)

cellnum
AreaFract
elevation
Pfactor
finished writing snow parameter file: /Users/diana/Dropbox/UW/Research/rasm/25_km/snow_params_25km.txt


In [19]:
import xarray as xr
u = xr.open_dataset(ncparams_25km)

In [30]:
u.root_fract.values.shape

(3, 12, 413, 551)

In [31]:
u.root_depth.values.shape

(3, 12, 413, 551)

In [32]:
u.LAI.values.shape

(12, 12, 413, 551)

In [28]:
rootzones = 3
for k in range(rootzones):
    print(k)

0
1
2


In [37]:
data['root_depth'].shape[1]

12